# Other Recommendation Model Types

So far we've used Amazon Personalize's `HRNN` model to deploy a `user->items` recommendation model - probably the first use case that comes to mind when thinking about recommendation engines.

...But there are two other use cases Personalize can help with on our website:

1. Recommending items **in the context of an item**: i.e. "Similar items", or more accurately "Users also bought/viewed/etc."
2. **Re-ranking a list of items** for relevance to a particular user: i.e. prioritizing impressions we want to make from a base set.

In this notebook, we'll use our previously imported data to train and deploy both types - powering further personalization on the website.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os
import json
import time

# External Dependencies:
import boto3

# Local Dependencies:
import util

%store -r role_arn
%store -r dataset_group_arn
%store -r hrnn_campaign_arn

personalize = boto3.client("personalize")

## "Similar" Items

Given a particular item in context (e.g. currently viewing the detail page for a product, or just added a product to the basket), we'd like to recommend related items that might drive additional transactions.

Of course our standard HRNN based user->item model is stateful and so will adapt to the user's session to an extent, but here we're aiming for a more explicit relationship from a certain item.

The model we train here will be deployed in the **"Similar Items"** section of the product detail page on our website - which also includes a **"Recommended For You"** section based on the user->item model, so we'll be able to see how the two compare in action.

### Creating the Solution Version

Our dataset(s) have already been loaded into Amazon Personalize for the previous model, so (as we did for HRNN) the first step is to create a Solution based on the `SIMS` recipe and train a Solution Version:

In [ ]:
sims_arn = "arn:aws:personalize:::recipe/aws-sims"

create_solution_response = personalize.create_solution(
    name=f"{os.environ['CF_STACK_NAME']}-soln-sims",
    datasetGroupArn=dataset_group_arn,
    recipeArn=sims_arn
)

sims_solution_arn = create_solution_response["solutionArn"]
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn=sims_solution_arn
)

sims_solution_version_arn = create_solution_version_response["solutionVersionArn"]
print(json.dumps(create_solution_version_response, indent=2))

In [ ]:
# Or if you'd like to resume with an existing solution version ARN taken from the console:
#sims_solution_version_arn = "arn:aws:personalize:us-east-1:387269085412:solution/thewsfooda-soln-sims/68311135"

In [ ]:
def solution_version_is_ready(description):
    status = description["solutionVersion"]["status"]
    if status == "ACTIVE":
        return True
    elif "FAILED" in status:
        raise ValueError(
            f"Wait ended with unexpected status '{status}':\n{description}"
        )
    else:
        return False

print(f"Waiting for solution version {sims_solution_version_arn}...")
util.progress.polling_spinner(
    lambda: personalize.describe_solution_version(solutionVersionArn=sims_solution_version_arn),
    solution_version_is_ready,
    fn_stringify_result = lambda desc: desc["solutionVersion"]["status"],
    timeout_secs=3*60*60,
)

### Solution metrics

As with HRNN, we can view metrics generated by Personalize to get an idea of the model's performance.

As with HRNN, these metrics are only indicative and the best way to understand business impact will be a live A/B test!

In [ ]:
solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn=sims_solution_version_arn
)

print(json.dumps(solution_metrics_response, indent=2))

### Deploying the solution

Once the solution is trained, we'll deploy it to a **Campaign** and then configure our website by setting a **Lambda Function environment variable**.

This time we're configuring the `LAMBDA_GETITEMRECS_ARN` function, as opposed to the `LAMBDA_GETRECS_ARN` function which powers user->item recommendations.

In [ ]:
create_campaign_response = personalize.create_campaign(
    name=f"{os.environ['CF_STACK_NAME']}-camp-sims",
    solutionVersionArn=sims_solution_version_arn,
    minProvisionedTPS=1,
)

sims_campaign_arn = create_campaign_response["campaignArn"]
%store sims_campaign_arn
print(json.dumps(create_campaign_response, indent=2))

In [ ]:
# Or if you'd like to resume with an existing campaign ARN taken from the console:
# sims_campaign_arn = "arn:aws:personalize:us-east-1:387269085412:campaign/thewsfooda-camp-sims"
# %store sims_campaign_arn

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours

print(f"Waiting for campaign {sims_campaign_arn}...")
details = {}
status = "UNKNOWN"
# TODO: Stricter status check
while status not in ("ACTIVE", "CREATE FAILED"):
    if status != "UNKNOWN":
        time.sleep(60)

    details = personalize.describe_campaign(campaignArn=sims_campaign_arn)
    status = details["campaign"]["status"]

if status != "ACTIVE":
    raise ValueError(f"Wait ended with unexpected status '{status}':\n{details}")
else:
    print("Done!")

In [ ]:
util.lambdafn.update_lambda_envvar(os.environ["LAMBDA_GETITEMRECS_ARN"], "CAMPAIGN_ARN", sims_campaign_arn)

You should now see recommendations in the "Similar Items" section of product detail pages on your website: How do they look? How do they differ from the standard HRNN-driven recommendations in the "Recommended for You" section of the same pages?

## Re-ranking items

For our final use case, we'd like to take a proposed list of items and **re-prioritize** them depending on what's expected to be most relevant to our user.

For an interactive demo on the website, we'll use this to promote certain items to the top of *search results*.

In practice search is a non-trivial use case for these models, because we need to strike a compromise between attending to the user's requests (as interpreted by the search engine in raw item relevance scores) and the personalization aspects (driven by the **history of interactions** rather than the current request). For demo purposes we take a simplified approach to this: promoting up to N results to the top of the list, and passing the remaining results through as ordered by the search engine.

**Other use cases** for this type of model could include prioritizing a shortlist of campaigns generated by rule-based or manual methods: e.g. selecting which campaigns/messages/items to show to each user from a list curated by marketing and segmented by demographics or other customer data.

### Creating the Solution Version

Again we'll create a solution and train a solution version, this time referencing the "Personalized Ranking" recipe:

In [ ]:
rerank_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

create_solution_response = personalize.create_solution(
    name=f"{os.environ['CF_STACK_NAME']}-soln-rerank",
    datasetGroupArn=dataset_group_arn,
    recipeArn=rerank_arn
)

rerank_solution_arn = create_solution_response["solutionArn"]
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn=rerank_solution_arn
)

rerank_solution_version_arn = create_solution_version_response["solutionVersionArn"]
print(json.dumps(create_solution_version_response, indent=2))

In [ ]:
# Or if you'd like to resume with an existing solution version ARN taken from the console:
#rerank_solution_version_arn = "arn:aws:personalize:us-east-1:387269085412:solution/thewsfooda-soln-rerank/68311135"

In [ ]:
print(f"Waiting for solution version {rerank_solution_version_arn}...")
util.progress.polling_spinner(
    lambda: personalize.describe_solution_version(solutionVersionArn=rerank_solution_version_arn),
    solution_version_is_ready,
    fn_stringify_result = lambda desc: desc["solutionVersion"]["status"],
    timeout_secs=3*60*60,
)

### Solution metrics

...You know the drill by now:

In [ ]:
solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn=rerank_solution_version_arn
)

print(json.dumps(solution_metrics_response, indent=2))

### Deploying the solution

Just like the other two models, but this time we're configuring the `LAMBDA_RERANK_ARN`:

In [ ]:
create_campaign_response = personalize.create_campaign(
    name=f"{os.environ['CF_STACK_NAME']}-camp-rerank",
    solutionVersionArn=rerank_solution_version_arn,
    minProvisionedTPS=1,
)

rerank_campaign_arn = create_campaign_response["campaignArn"]
%store rerank_campaign_arn
print(json.dumps(create_campaign_response, indent=2))

In [ ]:
# Or if you'd like to resume with an existing campaign ARN taken from the console:
# rerank_campaign_arn = "arn:aws:personalize:us-east-1:387269085412:campaign/thewsfooda-camp-rerank"
# %store rerank_campaign_arn

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours

print(f"Waiting for campaign {rerank_campaign_arn}...")
details = {}
status = "UNKNOWN"
# TODO: Stricter status check
while status not in ("ACTIVE", "CREATE FAILED"):
    if status != "UNKNOWN":
        time.sleep(60)

    details = personalize.describe_campaign(campaignArn=rerank_campaign_arn)
    status = details["campaign"]["status"]

if status != "ACTIVE":
    raise ValueError(f"Wait ended with unexpected status '{status}':\n{details}")
else:
    print("Done!")

In [ ]:
util.lambdafn.update_lambda_envvar(os.environ["LAMBDA_RERANK_ARN"], "CAMPAIGN_ARN", rerank_campaign_arn)

*Search results* on your website should now be ranked differently, depending which user is "signed in" in the dropdown menu. How do the new result rankings compare to raw search results? How could you **balance** promoting items a user's history suggests they'd be interested in, with those most relevant to their current search?

## And that's it!

Using the same core dataset and with just slightly tweaked API calls, we've deployed two more kinds of recommendations on our website to drive engagement and revenue.

...But there's one big question we've not really covered so far: improving model quality. In the next notebook, we'll show how additional metadata and hyperparameter tuning can help.